<a href="https://colab.research.google.com/github/Anagha7777/myapp/blob/main/spam%20ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
data=pd.read_csv('/content/spam.csv',encoding='Latin-1')


FileNotFoundError: [Errno 2] No such file or directory: '/content/spam.csv'

In [ ]:
data.head()

In [ ]:
pd.set_option('display.max_columns',None)
data=data[['v1','v2']]
data.columns=['label','text']
data.head()


In [ ]:
import gensim
data['cleaned_text']=data['text'].apply(lambda x:gensim.utils.simple_preprocess(x))
data.head()

In [ ]:
data['label']=data['label'].map({'ham':0,'spam':1})
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['cleaned_text'],data['label'],test_size=0.2,random_state=42)

In [ ]:
w2vec_model=gensim.models.Word2Vec(X_train,min_count=1)

In [ ]:
w2vec_model.wv['go']

In [ ]:
w2vec_model.wv.most_similar('go')

In [ ]:
words_trained=w2vec_model.wv.index_to_key

In [ ]:
words_trained

In [ ]:
x_train_vec = np.array([np.array([w2vec_model.wv[i] for i in ls if i in words_trained])
 for ls in X_train],dtype=object)

In [ ]:
x_test_vec = np.array([np.array([w2vec_model.wv[i] for i in ls if i in words_trained])
 for ls in X_test],dtype=object)

In [ ]:
x_train_vec[0]

In [ ]:
x_train_vec_avg=[]
for v in x_train_vec:
  if v.size:
    x_train_vec_avg.append(v.mean(axis=0))
  else:
    x_train_vec_avg.append(np.zeros(100,dtype='float64'))


x_test_vec_avg=[]
for v in x_test_vec:
  if v.size:
    x_test_vec_avg.append(v.mean(axis=0))
  else:
    x_test_vec_avg.append(np.zeros(100,dtype='float64'))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf=RandomForestClassifier()
clf.fit(x_train_vec_avg,y_train)
y_pred=clf.predict(x_test_vec_avg)
accuracy_score(y_test,y_pred)